## Implementacje algorytmów
Naiwny (podpunkt 1.1):

In [1]:
def naive(text, pattern):
    result = []

    for i in range(len(text) - len(pattern) + 1):
        for j in range(len(pattern)):
            if pattern[j] != text[i + j]:
                # NIEDOPASOWANIE
                break
        else:
            # DOPASOWANIE
            result.append(i)

    return result


Automat skończony (podpunkt 1.2):

In [2]:
def transition_table(pattern):
    alphabet = {e for e in pattern}
    result = []

    for q in range(0, len(pattern) + 1):
        result.append({})

        for a in alphabet:
            k = min(len(pattern), q + 1)

            while True:
                # x[:k] - prefiks o długości k
                # x[-k:] - sufiks o długości k
                if k == 0 or pattern[:k] == (pattern[:q] + a)[-k:]:
                    break
                k -= 1

            result[q][a] = k
    return result


def automat(text, pattern):
    result = []
    table = transition_table(pattern)

    length = len(pattern)
    q = 0

    for i, t in enumerate(text):
        if t in table[q]:
            q = table[q][t]

            if q == length:
                # DOPASOWANIE
                result.append(i - length + 1)

        else:
            q = 0

    return result


Knutha-Morrisa-Pratta (podpunkt 1.3):

In [3]:
def prefix_function(pattern):
    pi = [0]
    k = 0

    for q in range(1, len(pattern)):
        while k > 0 and pattern[k] != pattern[q]:
            k = pi[k - 1]

        if pattern[k] == pattern[q]:
            k = k + 1

        pi.append(k)

    return pi


def kmp(text, pattern):
    result = []
    pi = prefix_function(pattern)

    length = len(pattern)
    q = 0

    for i, t in enumerate(text):
        while q > 0 and pattern[q] != text[i]:
            # NIEDOPASOWANIE
            q = pi[q - 1]

        if pattern[q] == text[i]:
            q = q + 1

        if q == length:
            # DOPASOWANIE
            result.append(i - length + 1)
            q = pi[q - 1]

    return result


## Pomiary

In [4]:
text = open("pan-tadeusz.txt", "r").read()


In [5]:
import time


Testy (podpunkt 2):

In [6]:
def measure(function):
    start = time.time()
    function()
    return time.time() - start


def measure_many(function, count):
    return sum([measure(function) for i in range(count)]) / count


def measure_all(text, pattern, count=5):
    print(
        "Naiwny zajmuje:\t\t", measure_many(lambda: naive(text, pattern), count), "s."
    )
    print(
        "Automat zajmuje:\t", measure_many(lambda: automat(text, pattern), count), "s."
    )
    print(
        "  W tym preprocessing:\t",
        measure_many(lambda: transition_table(pattern), count),
        "s.",
    )
    print("KMP zajmuje:\t\t", measure_many(lambda: kmp(text, pattern), count), "s.")
    print(
        "  W tym preprocessing:\t",
        measure_many(lambda: prefix_function(pattern), count),
        "s.",
    )


Znajdowanie wzorca "pan" we fragmencie (podpunkt 3):

In [7]:
print("Naiwny:")
print("\tPan: ", len(naive(text, "pan")), "\tPani: ", len(naive(text, "pani")))
print()

print("Automat:")
print("\tPan: ", len(automat(text, "pan")), "\tPani: ", len(automat(text, "pani")))
print()

print("KMP:")
print("\tPan: ", len(kmp(text, "pan")), "\tPani: ", len(kmp(text, "pani")))
print()


Naiwny:
	Pan:  401 	Pani:  100

Automat:
	Pan:  401 	Pani:  100

KMP:
	Pan:  401 	Pani:  100



Pomiar czasu dla wzorców "pan" i "pani" we fragmencie (podpunkt 4):

In [8]:
measure_all(text, "pan")


Naiwny zajmuje:		 0.12914628982543946 s.
Automat zajmuje:	 0.04792141914367676 s.
  W tym preprocessing:	 1.5115737915039062e-05 s.
KMP zajmuje:		 0.07387838363647461 s.
  W tym preprocessing:	 1.049041748046875e-06 s.


In [9]:
measure_all(text, "pani")


Naiwny zajmuje:		 0.1490077018737793 s.
Automat zajmuje:	 0.047906494140625 s.
  W tym preprocessing:	 2.6035308837890626e-05 s.
KMP zajmuje:		 0.08041000366210938 s.
  W tym preprocessing:	 1.2874603271484376e-06 s.


## Propozycje
Poniższy dobór tekstu i wzorca sprawia że algorytm naiwny wypada wielokrotnie wolniej (podpunkt 5):

In [10]:
t = "a" * 10000 + "b"
p = "a" * 100 + "b"

measure_all(t, p)


Naiwny zajmuje:		 0.10018291473388671 s.
Automat zajmuje:	 0.0031793594360351564 s.
  W tym preprocessing:	 0.0016825675964355468 s.
KMP zajmuje:		 0.003362417221069336 s.
  W tym preprocessing:	 3.0994415283203125e-05 s.


## Wnioski
Pewnie w większości zastosowań praktycznych nie zauważylibyśmy znacznej różnicy w czasie działania powyższych algorytmów bo wielkość wzorca jest zaniedbywalna w stosunku do wielkości tekstu. Nic nie stoi jednak na przeszkodzie żeby używać zawsze KMP, zwłaszcza że nie jest właściwie trudniejszy w implementacji oraz preprocessing nie wymaga praktycznie żadnych zasobów. Wyjątkiem może byłyby specyficzne sytuacje w których np wzorzec zmienia się w trakcie przeszukiwania ale to już nie jest ten sam problem. (podpunkt 6)